In [21]:
!pip install sumy
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

from sumy.utils import get_stop_words

from IPython.display import Markdown, display
from google.colab import drive
drive.mount('/content/drive')
import nltk
nltk.download('punkt')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [18]:
with open('/content/drive/MyDrive/Text.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [19]:

def generate_summary(text, sentences_count=1, language="english"):
    parser = PlaintextParser.from_string(text, Tokenizer(language))

    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, sentences_count=sentences_count)
    summary_list = [str(sentence) for sentence in summary]

    stop_words = get_stop_words(language)
    filtered_summary = " ".join(
        [word for word in summary_list[0].split() if word.lower() not in stop_words]
    )

    return filtered_summary

In [34]:
sentences_count = 6
filtered_summary = generate_summary(text, sentences_count, language="english")
with open("/content/drive/MyDrive/summary.txt", "w", encoding="utf-8") as file:
    file.write(filtered_summary)

In [35]:
display(Markdown(text))

The Orbiter Discovery, OV-103, is considered eligible for listing in the National Register of Historic Places (NRHP) in the context of the U.S. Space Shuttle Program (1969-2011) under Criterion A in the areas of Space Exploration and Transportation and under Criterion C in the area of Engineering. Because it has achieved significance within the past fifty years, Criteria Consideration G applies. Under Criterion A, Discovery is significant as the oldest of the three extant orbiter vehicles constructed for the Space Shuttle Program (SSP), the longest running American space program to date; she was the third of five orbiters built by NASA. Unlike the Mercury, Gemini, and Apollo programs, the SSP’s emphasis was on cost effectiveness and reusability, and eventually the construction of a space station. Including her maiden voyage (launched August 30, 1984), Discovery flew to space thirty-nine times, more than any of the other four orbiters; she was also the first orbiter to fly twenty missions. She had the honor of being chosen as the Return to Flight vehicle after both the Challenger and Columbia accidents. Discovery was the first shuttle to fly with the redesigned SRBs, a result of the Challenger accident, and the first shuttle to fly with the Phase II and Block I SSME. Discovery also carried the Hubble Space Telescope to orbit and performed two of the five servicing missions to the observatory. She flew the first and last dedicated Department of Defense (DoD) missions, as well as the first unclassified defense-related mission. In addition, Discovery was vital to the construction of the International Space Station (ISS); she flew thirteen of the thirty-seven total missions flown to the station by a U.S. Space Shuttle. She was the first orbiter to dock to the ISS, and the first to perform an exchange of a resident crew. Under Criterion C, Discovery is significant as a feat of engineering. According to Wayne Hale, a flight director from Johnson Space Center, the Space Shuttle orbiter represents a “huge technological leap from expendable rockets and capsules to a reusable, winged, hypersonic, cargo-carrying spacecraft.” Although her base structure followed a conventional aircraft design, she used advanced materials that both minimized her weight for cargo-carrying purposes and featured low thermal expansion ratios, which provided a stable base for her Thermal Protection System (TPS) materials. The Space Shuttle orbiter also featured the first reusable TPS; all previous spaceflight vehicles had a single-use, ablative heat shield. Other notable engineering achievements of the orbiter included the first reusable orbital propulsion system, and the first two-fault-tolerant Integrated Avionics System. As Hale stated, the Space Shuttle remains “the largest, fastest, winged hypersonic aircraft in history,” having regularly flown at twenty-five times the speed of sound.


In [36]:
display(Markdown(filtered_summary))


achieved significance past fifty years, Criteria Consideration applies.

In [30]:
print(f"Довжина вихідного тексту: {len(text)}")
print(f"Довжина анотації: {len(filtered_summary)}")

Довжина вихідного тексту: 2907
Довжина анотації: 71


In [37]:
import nltk

from heapq import nlargest

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
from IPython.display import Markdown, display

In [38]:
nltk.download("punkt", quiet=True)
nltk.download("stopwords", quiet=True)

True

In [39]:
import os
import requests

# Перевіряємо наявність папки corpora/stopwords у каталозі nltk_data
nltk_data_path = nltk.data.path[0]
stopwords_path = os.path.join(nltk_data_path, "corpora", "stopwords")
if not os.path.exists(stopwords_path):
    os.makedirs(stopwords_path)


# Завантажуємо стоп-слова для української мови та зберігаємо у файл
url = "https://gist.github.com/sebleier/554280#file-nltk-s-list-of-english-stopwords"
r = requests.get(url)

with open(os.path.join(stopwords_path, "english"), "wb") as f:
    f.write(r.content)


stop_words = set(stopwords.words("english"))

In [40]:
with open('/content/drive/MyDrive/Text.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [41]:
def sentence_similarity(sent1, sent2, stopwords=None):

    if stopwords is None:
        stopwords = set()

    # Токенізуємо речення та видаляємо стоп-слова

    words1 = [
        word.lower()
        for word in word_tokenize(sent1)
        if word.isalnum() and word.lower() not in stopwords
    ]  # створюємо список слів (без стоп-слів) із першого речення
    words2 = [
        word.lower()
        for word in word_tokenize(sent2)
        if word.isalnum() and word.lower() not in stopwords
    ]  # створюємо список слів (без стоп-слів) із першого речення

    all_words = list(
        set(words1 + words2)
    )  # створюємо список унікальних слів з двох речень

    # Створюємо вектори типу компонентами яких є  одиниці та нулі
    # якщо слово є в речення, то ставимо 1, інакше 0
    # Отримуємо щось таке [1, 0, 0, 1, ...]
    vector1 = [1 if word in words1 else 0 for word in all_words]
    vector2 = [1 if word in words2 else 0 for word in all_words]

    # розраховуємо косинусну відстань між двома векторами

    return 1 - cosine_distance(vector1, vector2)


def build_similarity_matrix(sentences, stop_words):

    similarity_matrix = np.zeros((len(sentences), len(sentences)))

    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                similarity_matrix[i][j] = sentence_similarity(
                    sentences[i], sentences[j], stop_words
                )

    return similarity_matrix


def generate_summary(text, num_sentences=3, stop_words=stop_words):

    summarize_text = []

    sentences = sent_tokenize(text)  # Токенізуємо текст

    sentence_similarity_matrix = build_similarity_matrix(sentences, stop_words)

    sentence_similarity_scores = np.array(
        [sum(row) for row in sentence_similarity_matrix]
    )

    # Використовуємо nlargest для вибору топ N речень за порядком зменшення схожості
    top_sentences_indices = nlargest(
        num_sentences,
        range(len(sentence_similarity_scores)),
        key=sentence_similarity_scores.__getitem__,
    )

    # Формуємо короткий зміст з вибраних речень
    for i in top_sentences_indices:
        summarize_text.append(sentences[i])

    return " ".join(summarize_text)

In [44]:
num_sentences = 3
stop_words = set(stopwords.words("english"))
summary = generate_summary(text, num_sentences, stop_words)
with open("/content/drive/MyDrive/summary2.txt", "w", encoding="utf-8") as file:
    file.write(summary)

In [45]:
display(Markdown(text))


The Orbiter Discovery, OV-103, is considered eligible for listing in the National Register of Historic Places (NRHP) in the context of the U.S. Space Shuttle Program (1969-2011) under Criterion A in the areas of Space Exploration and Transportation and under Criterion C in the area of Engineering. Because it has achieved significance within the past fifty years, Criteria Consideration G applies. Under Criterion A, Discovery is significant as the oldest of the three extant orbiter vehicles constructed for the Space Shuttle Program (SSP), the longest running American space program to date; she was the third of five orbiters built by NASA. Unlike the Mercury, Gemini, and Apollo programs, the SSP’s emphasis was on cost effectiveness and reusability, and eventually the construction of a space station. Including her maiden voyage (launched August 30, 1984), Discovery flew to space thirty-nine times, more than any of the other four orbiters; she was also the first orbiter to fly twenty missions. She had the honor of being chosen as the Return to Flight vehicle after both the Challenger and Columbia accidents. Discovery was the first shuttle to fly with the redesigned SRBs, a result of the Challenger accident, and the first shuttle to fly with the Phase II and Block I SSME. Discovery also carried the Hubble Space Telescope to orbit and performed two of the five servicing missions to the observatory. She flew the first and last dedicated Department of Defense (DoD) missions, as well as the first unclassified defense-related mission. In addition, Discovery was vital to the construction of the International Space Station (ISS); she flew thirteen of the thirty-seven total missions flown to the station by a U.S. Space Shuttle. She was the first orbiter to dock to the ISS, and the first to perform an exchange of a resident crew. Under Criterion C, Discovery is significant as a feat of engineering. According to Wayne Hale, a flight director from Johnson Space Center, the Space Shuttle orbiter represents a “huge technological leap from expendable rockets and capsules to a reusable, winged, hypersonic, cargo-carrying spacecraft.” Although her base structure followed a conventional aircraft design, she used advanced materials that both minimized her weight for cargo-carrying purposes and featured low thermal expansion ratios, which provided a stable base for her Thermal Protection System (TPS) materials. The Space Shuttle orbiter also featured the first reusable TPS; all previous spaceflight vehicles had a single-use, ablative heat shield. Other notable engineering achievements of the orbiter included the first reusable orbital propulsion system, and the first two-fault-tolerant Integrated Avionics System. As Hale stated, the Space Shuttle remains “the largest, fastest, winged hypersonic aircraft in history,” having regularly flown at twenty-five times the speed of sound.


In [46]:
display(Markdown(summary))


She was the first orbiter to dock to the ISS, and the first to perform an exchange of a resident crew. In addition, Discovery was vital to the construction of the International Space Station (ISS); she flew thirteen of the thirty-seven total missions flown to the station by a U.S. Space Shuttle. Under Criterion A, Discovery is significant as the oldest of the three extant orbiter vehicles constructed for the Space Shuttle Program (SSP), the longest running American space program to date; she was the third of five orbiters built by NASA.

In [47]:
print(f"Довжина вихідного тексту: {len(text)}")
print(f"Довжина анотації: {len(summary)}")

Довжина вихідного тексту: 2907
Довжина анотації: 542
